# Assignment 1: Geometry Definition
The geometry of the first assignment (1D FEM) will be based on an inductor with the core (``Ferroxcube E100/60/28``) shown in the figure below. This is an E core (half), with two core halves making up a complete core. 

To model the magnetostatic fields in this configuration, we take the domain as a horizontal line through the center of the two core halves. This is obviously an approximation: by taking the one-dimensional domain like this, we are implicitly assuming that the geometry is invariant in the upward direction (which is not entirely true).

<img src="figures/core_geometry_def.png" alt="Geometry Definition" width="400"/>   

The dimensions given in the figure are

| Dimension | Value     |
|:----------|-----------|
| A         | 100.30 mm |
| B         |  73.15 mm |
| C         |  27.50 mm |
| D         |  59.40 mm |
| E         |  46.85 mm |
| F         |  27.50 mm |

This core is made of the 3C92 material, which is a manganese-zinc (MnZn) ferrite with a high permeability $\mu_r = 1500$ and a high saturation flux density $B_{sat} = 540\,\mathrm{mT}$. It is suitable for power inductors operating up to $0.2\,\mathrm{MHz}$.

## Inductor Design
To determine the required values of current density, a simple inductor is designed using the core mentioned above. For an inductance of $1.1\,\mathrm{mH}$, $N = 15$ turns are required. To ensure that the core does not saturate, the maximum current (for a flux density of $B_{max} = 300\,\mathrm{mT}$) is
$$ i_{max} = \frac{N B_{max} A}{L} = 3.15\,\mathrm{A} $$
Taking into account the 15 turns and the total window area, the current density for this maximum current will be
$$ J = \frac{N i_{max}}{A_{window}} = 2.2 \cdot 10^{4}\,\mathrm{A/m^2} = 2.2 \,\mathrm{A/cm^2} $$

## Implementation in gmsh

In [2]:
# Simulation domain
x0 = -0.1;
x1 = 0.1;

# Core dimensions
a = 100.3e-3;
b = 73.15e-3;
c = 27.5e-3;

# Mesh density
lc1 = 0.1;    # At x0 and x1
lc2 = 1e-3;   # At the core

In [3]:
using gmsh

gmsh.finalize()
gmsh.initialize()

In [4]:
gmsh.option.setNumber("General.Terminal", 1)     # Enable printing information to terminal
gmsh.model.add("inductor")                       # Create a new model
geo = gmsh.model.geo;

# Points
geo.addPoint(x0, 0, 0, lc1, 1)
geo.addPoint(x1, 0, 0, lc1, 2)

geo.addPoint(-a/2, 0, 0, lc2, 3)
geo.addPoint(-b/2, 0, 0, lc2, 4)
geo.addPoint(-c/2, 0, 0, lc2, 5)
geo.addPoint(c/2, 0, 0, lc2, 6)
geo.addPoint(b/2, 0, 0, lc2, 7)
geo.addPoint(a/2, 0, 0, lc2, 8)

# Lines
geo.addLine(1, 3, 1)
geo.addLine(3, 4, 2)
geo.addLine(4, 5, 3)
geo.addLine(5, 6, 4)
geo.addLine(6, 7, 5)
geo.addLine(7, 8, 6)
geo.addLine(8, 2, 7)

# If the geometry is not synchronized before creating physical groups, a warning will be displayed:
#    "Unknown entity of dimension %d and tag %d in physical group %d"
# Synchronizing at this point ensures that the geometry data and model agree on the set of entities.
geo.synchronize() 

# Physical properties
gmsh.model.addPhysicalGroup(1, [1, 7], 1)       # Air
gmsh.model.addPhysicalGroup(1, [2, 4, 6], 2)    # Core
gmsh.model.addPhysicalGroup(1, [3], 3)          # Coil left
gmsh.model.addPhysicalGroup(1, [5], 4)          # Coil right
gmsh.model.addPhysicalGroup(0, [1, 2], 5)       # Dirichlet boundary condition at x0 and x1

gmsh.model.setPhysicalName(1, 1, "Air")
gmsh.model.setPhysicalName(1, 2, "Core")
gmsh.model.setPhysicalName(1, 3, "Coil left")
gmsh.model.setPhysicalName(1, 4, "Coil right")
gmsh.model.setPhysicalName(0, 5, "D") 

# Generate mesh
gmsh.model.mesh.generate(1)

gmsh.write("geo/coil_geo.msh")

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 60%] Meshing curve 5 (Line)
Info    : [ 80%] Meshing curve 6 (Line)
Info    : [ 90%] Meshing curve 7 (Line)
Info    : Done meshing 1D (Wall 0.0145791s, CPU 0s)
Info    : 114 nodes 121 elements
Info    : Writing 'geo/coil_geo.msh'...
Info    : Done writing 'geo/coil_geo.msh'


In [ ]:
gmsh.fltk.run()

## Material Definition
According to the core parameters and inductor design presented above, the material properties should be modeled as shown in the table.

| Domain       | ID | Permeability $\mu_r$ | Conductivity $\sigma\,\mathrm{[S/m]}$ | Current density $J_z\,\mathrm{[A/m^2]}$ |
|:-------------|----|----------------------|---------------------------------------|-----------------------------------------|
| Air          | 1  | 1                    | 0                                     | 0                                       |
| Core         | 2  | 1500                 | 0.2                                   | 0                                       |
| Coil (left)  | 3  | 1                    | 0                                     | $+J$                                    |
| Coil (right) | 4  | 1                    | 0                                     | $-J$                                    |

Note that in reality, the coil also has conductivity. However, we will not model it like this to prevent eddy currents from being induced into the windings. While it is possible to model this, each of the turns in the winding should then be drawn separately in the geometry.

In [4]:
# Current density
J = 2.2e4;
fsource(group_id) = J * (group_id == 3) - J * (group_id == 4);

# Permeability
mu0 = 4e-7 * pi; # Permeability of vacuum
mu_r = 1500;
fmu(group_id) = mu0 + (mu_r - 1) * mu0 * (group_id == 2);
fnu(group_id) = 1 / fmu(group_id);

# Conductivity
sigma_core = 0.2;
fsigma(group_id) = sigma_core * (group_id == 2);